In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [12]:
batting_summary = pd.read_csv('datasets/fact_bating_summary.csv')

In [13]:
batting_summary.head()

,match_id,match,teamInnings,battingPos,batsmanName,out/not_out,runs,balls,4s,6s,SR
0,T203817,Super Kings Vs KKR,Super Kings,1,RuturajGaikwad,out,32,27,3,1,118.51
1,T203817,Super Kings Vs KKR,Super Kings,2,FafduPlessis,out,86,59,7,3,145.76
2,T203817,Super Kings Vs KKR,Super Kings,3,RobinUthappa,out,31,15,0,3,206.66
3,T203817,Super Kings Vs KKR,Super Kings,4,MoeenAli,not_out,37,20,2,3,185.00
4,T203817,Super Kings Vs KKR,KKR,1,ShubmanGill,out,51,43,6,0,118.60


### Top 10 batsmen based on past 3 years total runs scored.

In [14]:
top_10_batsman = batting_summary.groupby(['batsmanName']).agg({'runs': 'sum'}).nlargest(10, 'runs')
top_10_batsman

,runs
batsmanName,
ShubmanGill,1851
FafduPlessis,1831
RuturajGaikwad,1593
KLRahul,1516
JosButtler,1509
ShikharDhawan,1392
ViratKohli,1385
SanjuSamson,1304
SuryakumarYadav,1225


### Top 10 batsmen based on past 3 years batting average. (min 60 balls faced in each season)

In [15]:
match_summary = pd.read_csv('datasets/dim_match_summary.csv')
match_summary.head()

,team1,team2,winner,margin,matchDate,match_id
0,Super Kings,KKR,Super Kings,27 runs,15-Oct-21,T203817
1,Capitals,KKR,KKR,3 wickets,13-Oct-21,T206442
2,RCB,KKR,KKR,4 wickets,11-Oct-21,T208597
3,Capitals,Super Kings,Super Kings,4 wickets,10-Oct-21,T201819
4,Capitals,RCB,RCB,7 wickets,8-Oct-21,T205942


In [16]:
match_summary['matchDate'] = pd.to_datetime(match_summary['matchDate'], format='%d-%b-%y')
match_summary.head()

,team1,team2,winner,margin,matchDate,match_id
0,Super Kings,KKR,Super Kings,27 runs,2021-10-15,T203817
1,Capitals,KKR,KKR,3 wickets,2021-10-13,T206442
2,RCB,KKR,KKR,4 wickets,2021-10-11,T208597
3,Capitals,Super Kings,Super Kings,4 wickets,2021-10-10,T201819
4,Capitals,RCB,RCB,7 wickets,2021-10-08,T205942


In [17]:
match_summary['year'] = match_summary['matchDate'].dt.year
batting_match_merged_df = pd.merge(batting_summary, match_summary, on='match_id')

In [18]:
batting_match_merged_df.head()

,match_id,match,teamInnings,battingPos,batsmanName,out/not_out,runs,balls,4s,6s,SR,team1,team2,winner,margin,matchDate,year
0,T203817,Super Kings Vs KKR,Super Kings,1,RuturajGaikwad,out,32,27,3,1,118.51,Super Kings,KKR,Super Kings,27 runs,2021-10-15,2021
1,T203817,Super Kings Vs KKR,Super Kings,2,FafduPlessis,out,86,59,7,3,145.76,Super Kings,KKR,Super Kings,27 runs,2021-10-15,2021
2,T203817,Super Kings Vs KKR,Super Kings,3,RobinUthappa,out,31,15,0,3,206.66,Super Kings,KKR,Super Kings,27 runs,2021-10-15,2021
3,T203817,Super Kings Vs KKR,Super Kings,4,MoeenAli,not_out,37,20,2,3,185.00,Super Kings,KKR,Super Kings,27 runs,2021-10-15,2021
4,T203817,Super Kings Vs KKR,KKR,1,ShubmanGill,out,51,43,6,0,118.60,Super Kings,KKR,Super Kings,27 runs,2021-10-15,2021


In [19]:
batting_match_merged_df['SR'] = pd.to_numeric(batting_match_merged_df['SR'], errors='coerce')
batting_match_merged_df['SR'] = batting_match_merged_df['SR'].astype(float)

In [20]:
batting_yearly_stats = batting_match_merged_df.groupby(['batsmanName', 'year']).agg({
    'teamInnings': 'count',
    'balls': 'sum',
    '4s' : 'sum',
    '6s' : 'sum',
    'runs': 'sum',
    'SR': 'mean',
    'out/not_out': lambda x: (x == 'not_out').sum()
}).reset_index()

In [21]:
batting_yearly_stats.head()

,batsmanName,year,teamInnings,balls,4s,6s,runs,SR,out/not_out
0,ABdeVilliers,2021,14,211,23,16,313,133.299286,4
1,AaronFinch,2022,5,61,10,3,86,109.426000,0
2,AbdulBasith,2023,1,1,0,0,1,100.000000,1
3,AbdulSamad,2021,10,87,4,8,111,100.077000,1
4,AbdulSamad,2022,2,7,0,0,4,33.330000,0


In [12]:
players_all_years = batting_yearly_stats['batsmanName'].value_counts()[batting_yearly_stats['batsmanName'].value_counts() == 3].index
batting_min_60balls_stats = batting_yearly_stats[batting_yearly_stats['batsmanName'].isin(players_all_years)] 
batting_min_60balls_stats = batting_min_60balls_stats.groupby('batsmanName').filter(lambda x: (x['balls'] >= 60).all())
batting_min_60balls_stats.head()

,batsmanName,year,teamInnings,balls,4s,6s,runs,SR,out/not_out
9,AbhishekSharma,2021,9,139,17,8,188,121.544444,1
10,AbhishekSharma,2022,11,250,37,9,331,109.323636,0
11,AbhishekSharma,2023,11,157,28,6,226,111.402727,0
16,AidenMarkram,2021,9,165,14,7,201,119.840000,1
17,AidenMarkram,2022,9,228,23,16,326,141.247778,4


In [13]:
batting_groupby_batsman = batting_min_60balls_stats.groupby('batsmanName')
batting_3yrs_stats = batting_groupby_batsman.agg({
    'teamInnings': 'sum',
    'balls': 'sum',
    '4s' : 'sum',
    '6s' : 'sum',
    'runs': 'sum',
    'SR': 'mean',
    'out/not_out': 'sum'
}).reset_index()
batting_3yrs_stats.head()

,batsmanName,teamInnings,balls,4s,6s,runs,SR,out/not_out
0,AbhishekSharma,31,546,82,23,745,114.090269,1
1,AidenMarkram,31,590,55,31,775,124.404644,7
2,AmbatiRayudu,36,507,51,41,689,116.461111,5
3,AndreRussell,35,468,45,64,745,132.169476,8
4,AxarPatel,29,369,35,26,505,126.531571,10


In [14]:
batting_3yrs_stats['batting_average'] = batting_3yrs_stats['runs'] / (batting_3yrs_stats['teamInnings'] - batting_3yrs_stats['out/not_out'])
batting_3yrs_stats.head()

,batsmanName,teamInnings,balls,4s,6s,runs,SR,out/not_out,batting_average
0,AbhishekSharma,31,546,82,23,745,114.090269,1,24.833333
1,AidenMarkram,31,590,55,31,775,124.404644,7,32.291667
2,AmbatiRayudu,36,507,51,41,689,116.461111,5,22.225806
3,AndreRussell,35,468,45,64,745,132.169476,8,27.592593
4,AxarPatel,29,369,35,26,505,126.531571,10,26.578947


In [15]:
top_10_batsmen_avg = batting_3yrs_stats.nlargest(10, 'batting_average')
top_10_batsmen_avg[['batsmanName', 'batting_average']]

,batsmanName,batting_average
15,KLRahul,50.533333
10,FafduPlessis,43.595238
5,DavidMiller,43.200000
14,JosButtler,41.916667
35,ShimronHetmyer,40.666667
37,ShubmanGill,40.239130
34,ShikharDhawan,39.771429
32,RuturajGaikwad,37.928571
6,DavidWarner,37.900000
38,SuryakumarYadav,35.000000


### Top 10 batsmen based on past 3 years strike rate (min 60 balls faced in each season)

In [16]:
top_10_batsmen_sr = batting_3yrs_stats.nlargest(10, 'SR')
top_10_batsmen_sr[['batsmanName', 'SR']]

,batsmanName,SR
11,GlennMaxwell,154.340053
35,ShimronHetmyer,143.628131
29,RavindraJadeja,141.882596
38,SuryakumarYadav,137.959940
9,DineshKarthik,136.671987
17,LiamLivingstone,136.596852
18,MShahrukhKhan,134.254874
28,RahulTripathi,132.716181
3,AndreRussell,132.169476
33,SanjuSamson,129.844229


### Top 10 batsmen based on past 3 years boundary % (fours and sixes).

In [17]:
batting_3yrs_stats['boundary_percentage'] = ((4 * batting_3yrs_stats['4s']) + (6 * batting_3yrs_stats['6s'])*100) / batting_3yrs_stats['runs']
batting_3yrs_stats.head()

,batsmanName,teamInnings,balls,4s,6s,runs,SR,out/not_out,batting_average,boundary_percentage
0,AbhishekSharma,31,546,82,23,745,114.090269,1,24.833333,18.963758
1,AidenMarkram,31,590,55,31,775,124.404644,7,32.291667,24.283871
2,AmbatiRayudu,36,507,51,41,689,116.461111,5,22.225806,36.000000
3,AndreRussell,35,468,45,64,745,132.169476,8,27.592593,51.785235
4,AxarPatel,29,369,35,26,505,126.531571,10,26.578947,31.168317


In [18]:
top_10_batsmen_boundary_percentage = batting_3yrs_stats.nlargest(10, 'boundary_percentage')
top_10_batsmen_boundary_percentage[['batsmanName', 'boundary_percentage']]

,batsmanName,boundary_percentage
3,AndreRussell,51.785235
24,NicholasPooran,43.045267
17,LiamLivingstone,43.018470
18,MShahrukhKhan,39.680751
35,ShimronHetmyer,39.587822
36,ShivamDube,39.282818
20,MarcusStoinis,38.418605
2,AmbatiRayudu,36.000000
22,MitchellMarsh,33.562005
11,GlennMaxwell,33.469522


In [95]:
batting_3yrs_stats.to_csv('batting_3yrs_stats.csv')

### Top 10 bowlers based on past 3 years total wickets taken.

In [185]:
bowling_summary = pd.read_csv('datasets/fact_bowling_summary.csv')
bowling_summary.head()

,match_id,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls
0,T203817,Super Kings Vs KKR,KKR,ShakibAlHasan,3.0,0,33,0,11.0,6,2,3,0,0
1,T203817,Super Kings Vs KKR,KKR,ShivamMavi,4.0,0,32,1,8.0,8,1,2,0,0
2,T203817,Super Kings Vs KKR,KKR,LockieFerguson,4.0,0,56,0,14.0,4,7,2,2,0
3,T203817,Super Kings Vs KKR,KKR,VarunChakravarthy,4.0,0,38,0,9.5,5,2,2,0,1
4,T203817,Super Kings Vs KKR,KKR,SunilNarine,4.0,0,26,2,6.5,6,0,1,1,0


In [186]:
top_10_bowlers_wickets = bowling_summary.groupby(['bowlerName']).agg({'wickets': 'sum'}).nlargest(10, 'wickets')
top_10_bowlers_wickets

,wickets
bowlerName,
MohammedShami,67
YuzvendraChahal,66
HarshalPatel,65
RashidKhan,63
AveshKhan,47
ArshdeepSingh,45
KagisoRabada,45
VarunChakravarthy,44
ShardulThakur,43


### Top 10 bowlers based on past 3 years economy rate. (min 60 balls bowled in each season)

In [187]:
bowling_summary['balls'] = bowling_summary['overs'].apply(lambda x: int(str(x).split('.')[0]) * 6 + int(str(x).split('.')[1]))

In [188]:
bowling_match_merged_df = pd.merge(bowling_summary, match_summary, on='match_id')
bowling_match_merged_df.head()

,match_id,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,...,6s,wides,noBalls,balls,team1,team2,winner,margin,matchDate,year
0,T203817,Super Kings Vs KKR,KKR,ShakibAlHasan,3.0,0,33,0,11.0,6,...,3,0,0,18,Super Kings,KKR,Super Kings,27 runs,2021-10-15,2021
1,T203817,Super Kings Vs KKR,KKR,ShivamMavi,4.0,0,32,1,8.0,8,...,2,0,0,24,Super Kings,KKR,Super Kings,27 runs,2021-10-15,2021
2,T203817,Super Kings Vs KKR,KKR,LockieFerguson,4.0,0,56,0,14.0,4,...,2,2,0,24,Super Kings,KKR,Super Kings,27 runs,2021-10-15,2021
3,T203817,Super Kings Vs KKR,KKR,VarunChakravarthy,4.0,0,38,0,9.5,5,...,2,0,1,24,Super Kings,KKR,Super Kings,27 runs,2021-10-15,2021
4,T203817,Super Kings Vs KKR,KKR,SunilNarine,4.0,0,26,2,6.5,6,...,1,1,0,24,Super Kings,KKR,Super Kings,27 runs,2021-10-15,2021


In [189]:
bowling_match_merged_df[bowling_summary['bowlerName'] == 'DewaldBrevis']

,match_id,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,...,6s,wides,noBalls,balls,team1,team2,winner,margin,matchDate,year
1343,T204746,Mumbai Vs RCB,Mumbai,DewaldBrevis,0.3,0,8,1,16.0,1,...,0,0,0,3,Mumbai,RCB,RCB,7 wickets,2022-04-09,2022


In [190]:
bowling_yearly_stats = bowling_match_merged_df.groupby(['bowlerName', 'year']).agg({
    'balls': 'sum',
    'maiden': 'sum',
    'runs' : 'sum',
    'wickets' : 'sum',
    'economy': 'mean',
    '0s': 'sum',
}).reset_index()

In [191]:
bowling_yearly_stats.head()

,bowlerName,year,balls,maiden,runs,wickets,economy,0s
0,AbdulSamad,2021,6,0,9,1,9.000,1
1,AbdulSamad,2022,6,0,8,0,8.000,1
2,AbhishekSharma,2021,60,0,60,4,6.250,24
3,AbhishekSharma,2022,18,0,28,0,10.750,3
4,AbhishekSharma,2023,72,0,124,2,10.498,23


In [192]:
players_all_years = bowling_yearly_stats['bowlerName'].value_counts()[bowling_yearly_stats['bowlerName'].value_counts() == 3].index
bowling_stats = bowling_yearly_stats[bowling_yearly_stats['bowlerName'].isin(players_all_years)] 
bowling_stats = bowling_stats.groupby('bowlerName').filter(lambda x: (x['balls'] >= 60).all())
bowling_stats.head()

,bowlerName,year,balls,maiden,runs,wickets,economy,0s
25,AndreRussell,2021,156,0,255,14,9.782000,54
26,AndreRussell,2022,127,0,211,14,9.231818,43
27,AndreRussell,2023,91,0,172,7,13.228889,32
29,AnrichNortje,2021,254,1,292,17,6.879091,114
30,AnrichNortje,2022,62,0,112,4,11.416667,19


In [193]:
bowling_groupby_bowlername = bowling_stats.groupby('bowlerName')
bowling_3yrs_stats = bowling_groupby_bowlername.agg({
    'balls': 'sum',
    'maiden': 'sum',
    'runs' : 'sum',
    'wickets' : 'sum',
    'economy': 'mean',
    '0s': 'sum',
}).reset_index()
bowling_3yrs_stats.head()

,bowlerName,balls,maiden,runs,wickets,economy,0s
0,AndreRussell,374,0,638,35,10.747569,129
1,AnrichNortje,556,1,768,31,9.131919,229
2,ArshdeepSingh,853,0,1220,45,8.699022,312
3,AveshKhan,802,1,1115,47,8.740760,334
4,AxarPatel,792,0,939,30,7.287077,279


In [194]:
bowling_3yrs_stats['economy'] = (bowling_3yrs_stats['runs'] / bowling_3yrs_stats['balls']) * 6

In [195]:
top_10_bowlers_economy = bowling_3yrs_stats.nsmallest(10, 'economy')
top_10_bowlers_economy[['bowlerName', 'economy']]

,bowlerName,economy
29,SunilNarine,6.600000
17,MoeenAli,7.039813
20,MohsinKhan,7.069767
4,AxarPatel,7.113636
22,RashidKhan,7.203282
13,KrunalPandya,7.449190
25,RavindraJadeja,7.455882
24,RavichandranAshwin,7.500000
33,VarunChakravarthy,7.572025
21,RahulChahar,7.629187


### Top 10 bowlers based on past 3 years bowling average. (min 60 balls bowled in each season)

In [196]:
bowling_3yrs_stats['bowling_avg'] = bowling_3yrs_stats['runs'] / bowling_3yrs_stats['wickets']

In [197]:
top_10_bowlers_avg = bowling_3yrs_stats.nsmallest(10, 'bowling_avg')
top_10_bowlers_avg[['bowlerName', 'bowling_avg']]

,bowlerName,bowling_avg
20,MohsinKhan,17.882353
0,AndreRussell,18.228571
37,YuzvendraChahal,20.196970
9,HarshalPatel,20.353846
22,RashidKhan,20.904762
18,MohammedShami,20.970149
34,WaninduHasaranga,21.371429
3,AveshKhan,23.723404
11,KagisoRabada,23.755556
17,MoeenAli,23.857143


### Top 5 bowlers based on past 3 years dot ball %

In [200]:
bowling_stats[bowling_stats['bowlerName'] == 'DewaldBrevis']

,bowlerName,balls,maiden,runs,wickets,economy,0s
39,DewaldBrevis,3,0,8,1,16.0,1


In [203]:
bowling_3yrs_stats['dot_percentage'] = (bowling_3yrs_stats['0s'] / bowling_3yrs_stats['balls'])*100

In [204]:
top_10_bowler_dot = bowling_3yrs_stats.nlargest(10, 'dot_percentage')
top_10_bowler_dot[['bowlerName', 'dot_percentage']]

,bowlerName,dot_percentage
20,MohsinKhan,50.387597
19,MohammedSiraj,47.712418
18,MohammedShami,47.574627
31,TrentBoult,46.365639
32,UmranMalik,44.147844
12,KhaleelAhmed,43.607706
11,KagisoRabada,43.413978
16,MarcoJansen,42.564103
5,BhuvneshwarKumar,42.479908
8,HardikPandya,41.741742


In [205]:
match_summary['won_by_chase'] = match_summary['margin'].apply(lambda x: True if 'wickets' in x.lower() else False)
match_summary.head()

,team1,team2,winner,margin,matchDate,match_id,year,won_by_chase
0,Super Kings,KKR,Super Kings,27 runs,2021-10-15,T203817,2021,False
1,Capitals,KKR,KKR,3 wickets,2021-10-13,T206442,2021,True
2,RCB,KKR,KKR,4 wickets,2021-10-11,T208597,2021,True
3,Capitals,Super Kings,Super Kings,4 wickets,2021-10-10,T201819,2021,True
4,Capitals,RCB,RCB,7 wickets,2021-10-08,T205942,2021,True


In [207]:
win_counts = match_summary['winner'].value_counts()
win_counts

Super Kings     25
RCB             25
Titans          23
Royals          22
KKR             21
Capitals        21
Mumbai          20
Punjab Kings    19
Super Giants    17
Sunrisers       13
Name: winner, dtype: int64

In [209]:
total_matches_played = match_summary['team1'].append(match_summary['team2']).value_counts()
total_matches_played

C:\Users\mgkam\AppData\Local\Temp\ipykernel_46856\442234189.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_matches_played = match_summary['team1'].append(match_summary['team2']).value_counts()


Super Kings     45
RCB             45
KKR             45
Royals          45
Mumbai          44
Capitals        43
Punjab Kings    42
Sunrisers       41
Titans          33
Super Giants    29
dtype: int64

In [211]:
win_percentage = (win_counts / total_matches_played) * 100
win_percentage.sort_values(ascending=False)

Titans          69.696970
Super Giants    58.620690
RCB             55.555556
Super Kings     55.555556
Royals          48.888889
Capitals        48.837209
KKR             46.666667
Mumbai          45.454545
Punjab Kings    45.238095
Sunrisers       31.707317
dtype: float64

In [215]:
wins_by_chasing = match_summary[match_summary['margin'].str.contains('wickets', case=False)]
chasing_wins_count = wins_by_chasing['winner'].value_counts()
chasing_wins_count

KKR             14
Capitals        14
Titans          14
Mumbai          13
Super Kings     11
RCB             10
Royals           9
Sunrisers        9
Punjab Kings     8
Super Giants     4
Name: winner, dtype: int64

In [214]:
wins_by_defending = match_summary[match_summary['margin'].str.contains('runs', case=False)]
defending_wins_count = wins_by_defending['winner'].value_counts()
defending_wins_count

Super Kings     14
RCB             14
Royals          13
Punjab Kings    11
Super Giants    11
Titans           9
Mumbai           7
KKR              7
Capitals         7
Sunrisers        4
Name: winner, dtype: int64

### Orange Cap Predictions

In [130]:
batting_consistency_merged_df.head()

,batsmanName,teamInnings,balls,4s,6s,runs_x,SR,out/not_out,consistency_factor
0,AbhinavManohar,13,158,20,10,222,118.142500,1,0.584348
1,AbhishekSharma,31,546,82,23,745,114.090269,1,0.424465
2,AidenMarkram,31,590,55,31,775,124.404644,7,0.540019
3,AmbatiRayudu,36,507,51,41,689,116.461111,5,0.502401
4,AndreRussell,35,468,45,64,745,132.169476,8,0.488124


In [29]:
batting_consistency_merged_df['balls_per_match'] = (batting_consistency_merged_df['balls'] / batting_consistency_merged_df['teamInnings'])

In [30]:
batting_consistency_merged_df['runs_per_match'] = (batting_consistency_merged_df['balls_per_match'] * batting_consistency_merged_df['SR'])/100

In [31]:
batting_consistency_merged_df['expected_runs'] = batting_consistency_merged_df['runs_per_match'] * 14

In [32]:
batting_consistency_merged_df.nlargest(10, 'expected_runs')

,batsmanName,teamInnings,balls,4s,6s,runs_x,SR,out/not_out,runs_y,balls_per_match,runs_per_match,expected_runs
22,FafduPlessis,14,475,60,36,730,160.207857,1,0.241461,33.928571,54.356237,760.987321
69,ShubmanGill,17,564,85,33,890,136.261765,2,0.182610,33.176471,45.206844,632.895820
61,SaiSudharsan,8,256,33,12,362,133.975000,1,0.343721,32.000000,42.872000,600.208000
26,HeinrichKlaasen,11,253,32,25,448,182.172727,2,0.319339,23.000000,41.899727,586.596182
76,ViratKohli,14,457,65,16,639,118.433571,2,0.275410,32.642857,38.660102,541.241421
19,DevonConway,15,481,77,18,672,117.232667,2,0.136574,32.066667,37.592608,526.296518
79,YashasviJaiswal,14,382,82,26,625,136.939286,1,0.231896,27.285714,37.364862,523.108071
60,RuturajGaikwad,15,400,46,30,590,139.366667,1,0.237644,26.666667,37.164444,520.302222
28,JasonRoy,8,188,36,12,285,145.907500,0,0.463342,23.500000,34.288263,480.035675
51,QuintondeKock,4,102,10,8,143,133.430000,0,0.201128,25.500000,34.024650,476.345100


In [24]:
batsman_stats = batting_summary.groupby('batsmanName')['runs'].std()

In [25]:
max_std = batsman_stats.max()
max_std

38.26077512582591

In [26]:
batting_summary.head()

,match_id,match,teamInnings,battingPos,batsmanName,out/not_out,runs,balls,4s,6s,SR
0,T203817,Super Kings Vs KKR,Super Kings,1,RuturajGaikwad,out,32,27,3,1,118.51
1,T203817,Super Kings Vs KKR,Super Kings,2,FafduPlessis,out,86,59,7,3,145.76
2,T203817,Super Kings Vs KKR,Super Kings,3,RobinUthappa,out,31,15,0,3,206.66
3,T203817,Super Kings Vs KKR,Super Kings,4,MoeenAli,not_out,37,20,2,3,185.00
4,T203817,Super Kings Vs KKR,KKR,1,ShubmanGill,out,51,43,6,0,118.60


In [27]:
batsman_stats_normalized = 1 - (batsman_stats / max_std)
batsman_stats_normalized.fillna(0, inplace=True)
batsman_stats_normalized

batsmanName
ABdeVilliers       0.323817
AaronFinch         0.393376
AbdulBasith        0.000000
AbdulSamad         0.689751
AbhijeetTomar      0.000000
                     ...   
YashDayal          0.000000
YashDhull          0.825975
YashasviJaiswal    0.231896
YudhvirSingh       0.981519
YuzvendraChahal    0.908523
Name: runs, Length: 264, dtype: float64

In [22]:
batsmen_2023 = batting_yearly_stats[batting_yearly_stats['year'] == 2023]['batsmanName'].unique()
batting_filtered_stats = batting_yearly_stats[(batting_yearly_stats['batsmanName'].isin(batsmen_2023)) & (batting_yearly_stats['year'] == 2023) & (batting_yearly_stats['balls'] >= 60)]
batting_filtered_stats.head()

,batsmanName,year,teamInnings,balls,4s,6s,runs,SR,out/not_out
5,AbdulSamad,2023,8,128,10,11,169,144.896250,4
8,AbhinavManohar,2023,6,83,6,7,114,131.355000,0
11,AbhishekSharma,2023,11,157,28,6,226,111.402727,0
18,AidenMarkram,2023,13,197,18,8,248,112.126154,2
21,AjinkyaRahane,2023,11,189,24,16,326,161.448182,1


In [23]:
batting_groupby_batsman = batting_filtered_stats.groupby('batsmanName')
batting_3yrs_stats = batting_groupby_batsman.agg({
    'teamInnings': 'sum',
    'balls': 'sum',
    '4s' : 'sum',
    '6s' : 'sum',
    'runs': 'sum',
    'SR': 'mean',
    'out/not_out': 'sum'
}).reset_index()
batting_3yrs_stats.head()

,batsmanName,teamInnings,balls,4s,6s,runs,SR,out/not_out
0,AbdulSamad,8,128,10,11,169,144.896250,4
1,AbhinavManohar,6,83,6,7,114,131.355000,0
2,AbhishekSharma,11,157,28,6,226,111.402727,0
3,AidenMarkram,13,197,18,8,248,112.126154,2
4,AjinkyaRahane,11,189,24,16,326,161.448182,1


In [28]:
batting_consistency_merged_df = pd.merge(batting_3yrs_stats, batsman_stats_normalized, on='batsmanName')
batting_consistency_merged_df.head()

,batsmanName,teamInnings,balls,4s,6s,runs_x,SR,out/not_out,runs_y
0,AbdulSamad,8,128,10,11,169,144.896250,4,0.689751
1,AbhinavManohar,6,83,6,7,114,131.355000,0,0.584348
2,AbhishekSharma,11,157,28,6,226,111.402727,0,0.424465
3,AidenMarkram,13,197,18,8,248,112.126154,2,0.540019
4,AjinkyaRahane,11,189,24,16,326,161.448182,1,0.529922


In [33]:
batting_consistency_merged_df = batting_consistency_merged_df.rename(columns={'runs_y': 'consistency_factor'})
batting_consistency_merged_df.head()

,batsmanName,teamInnings,balls,4s,6s,runs_x,SR,out/not_out,consistency_factor,balls_per_match,runs_per_match,expected_runs
0,AbdulSamad,8,128,10,11,169,144.896250,4,0.689751,16.000000,23.183400,324.567600
1,AbhinavManohar,6,83,6,7,114,131.355000,0,0.584348,13.833333,18.170775,254.390850
2,AbhishekSharma,11,157,28,6,226,111.402727,0,0.424465,14.272727,15.900207,222.602904
3,AidenMarkram,13,197,18,8,248,112.126154,2,0.540019,15.153846,16.991425,237.879948
4,AjinkyaRahane,11,189,24,16,326,161.448182,1,0.529922,17.181818,27.739733,388.356263


In [34]:
batting_consistency_merged_df['consistent_expected_runs'] = (batting_consistency_merged_df['expected_runs'] * batting_consistency_merged_df['consistency_factor'])

In [35]:
batting_consistency_merged_df.head()

,batsmanName,teamInnings,balls,4s,6s,runs_x,SR,out/not_out,consistency_factor,balls_per_match,runs_per_match,expected_runs,consistent_expected_runs
0,AbdulSamad,8,128,10,11,169,144.896250,4,0.689751,16.000000,23.183400,324.567600,223.870953
1,AbhinavManohar,6,83,6,7,114,131.355000,0,0.584348,13.833333,18.170775,254.390850,148.652886
2,AbhishekSharma,11,157,28,6,226,111.402727,0,0.424465,14.272727,15.900207,222.602904,94.487110
3,AidenMarkram,13,197,18,8,248,112.126154,2,0.540019,15.153846,16.991425,237.879948,128.459612
4,AjinkyaRahane,11,189,24,16,326,161.448182,1,0.529922,17.181818,27.739733,388.356263,205.798648


In [45]:
batting_consistency_merged_df.nlargest(10, 'expected_runs')

,batsmanName,teamInnings,balls,4s,6s,runs_x,SR,out/not_out,consistency_factor,balls_per_match,runs_per_match,expected_runs,consistent_expected_runs,normalized_match_count
22,FafduPlessis,14,475,60,36,730,160.207857,1,0.241461,33.928571,54.356237,760.987321,151.322260,0.823529
69,ShubmanGill,17,564,85,33,890,136.261765,2,0.182610,33.176471,45.206844,632.895820,115.573289,1.000000
61,SaiSudharsan,8,256,33,12,362,133.975000,1,0.343721,32.000000,42.872000,600.208000,97.084163,0.470588
26,HeinrichKlaasen,11,253,32,25,448,182.172727,2,0.319339,23.000000,41.899727,586.596182,121.208976,0.647059
76,ViratKohli,14,457,65,16,639,118.433571,2,0.275410,32.642857,38.660102,541.241421,122.757941,0.823529
19,DevonConway,15,481,77,18,672,117.232667,2,0.136574,32.066667,37.592608,526.296518,63.422299,0.882353
79,YashasviJaiswal,14,382,82,26,625,136.939286,1,0.231896,27.285714,37.364862,523.108071,99.899811,0.823529
60,RuturajGaikwad,15,400,46,30,590,139.366667,1,0.237644,26.666667,37.164444,520.302222,109.100165,0.882353
28,JasonRoy,8,188,36,12,285,145.907500,0,0.463342,23.500000,34.288263,480.035675,104.668490,0.470588
51,QuintondeKock,4,102,10,8,143,133.430000,0,0.201128,25.500000,34.024650,476.345100,22.542629,0.235294


In [44]:
batting_consistency_merged_df.nlargest(10, 'consistent_expected_runs')

,batsmanName,teamInnings,balls,4s,6s,runs_x,SR,out/not_out,consistency_factor,balls_per_match,runs_per_match,expected_runs,consistent_expected_runs,normalized_match_count
57,RinkuSingh,14,317,31,29,474,144.611429,6,0.491025,22.642857,32.744159,458.418229,185.372060,0.823529
11,AxarPatel,13,203,21,15,283,143.928462,3,0.637472,15.615385,22.474983,314.649760,153.384912,0.764706
22,FafduPlessis,14,475,60,36,730,160.207857,1,0.241461,33.928571,54.356237,760.987321,151.322260,0.823529
29,JiteshSharma,14,198,22,21,309,148.360714,1,0.625124,14.142857,20.982444,293.754214,151.227053,0.823529
68,ShivamDube,14,264,12,35,418,153.422143,3,0.435235,18.857143,28.931033,405.034457,145.176018,0.823529
72,TilakVarma,11,209,26,23,343,158.990909,3,0.503236,19.000000,30.208273,422.915818,137.711227,0.647059
27,IshanKishan,15,318,54,18,454,128.024667,0,0.399465,21.200000,27.141229,379.977211,133.930280,0.882353
4,AjinkyaRahane,11,189,24,16,326,161.448182,1,0.529922,17.181818,27.739733,388.356263,133.163831,0.647059
71,SuryakumarYadav,16,334,65,28,605,155.473750,2,0.306537,20.875000,32.455145,454.372034,131.088902,0.941176
46,NitishRana,14,293,39,20,413,122.281429,1,0.442973,20.928571,25.591756,358.284586,130.702767,0.823529


In [38]:
match_count_max = batting_consistency_merged_df['teamInnings'].max()
match_count_max

17

In [39]:
batting_consistency_merged_df['normalized_match_count'] = batting_consistency_merged_df['teamInnings'] / match_count_max

In [47]:
batting_consistency_merged_df.head()

,batsmanName,teamInnings,balls,4s,6s,runs_x,SR,out/not_out,consistency_factor,balls_per_match,runs_per_match,expected_runs,consistent_expected_runs,normalized_match_count,expected_runs_innings
0,AbdulSamad,8,128,10,11,169,144.896250,4,0.689751,16.000000,23.183400,324.567600,105.351037,0.470588,152.737694
1,AbhinavManohar,6,83,6,7,114,131.355000,0,0.584348,13.833333,18.170775,254.390850,52.465725,0.352941,89.785006
2,AbhishekSharma,11,157,28,6,226,111.402727,0,0.424465,14.272727,15.900207,222.602904,61.138718,0.647059,144.037173
3,AidenMarkram,13,197,18,8,248,112.126154,2,0.540019,15.153846,16.991425,237.879948,98.233821,0.764706,181.908195
4,AjinkyaRahane,11,189,24,16,326,161.448182,1,0.529922,17.181818,27.739733,388.356263,133.163831,0.647059,251.289347


In [43]:
batting_consistency_merged_df['consistent_expected_runs'] = (batting_consistency_merged_df['expected_runs'] * batting_consistency_merged_df['consistency_factor'] * batting_consistency_merged_df['normalized_match_count'])

In [46]:
batting_consistency_merged_df['expected_runs_innings'] = (batting_consistency_merged_df['expected_runs'] * batting_consistency_merged_df['normalized_match_count'])

In [48]:
batting_consistency_merged_df.nlargest(10, 'expected_runs_innings')

,batsmanName,teamInnings,balls,4s,6s,runs_x,SR,out/not_out,consistency_factor,balls_per_match,runs_per_match,expected_runs,consistent_expected_runs,normalized_match_count,expected_runs_innings
69,ShubmanGill,17,564,85,33,890,136.261765,2,0.182610,33.176471,45.206844,632.895820,115.573289,1.000000,632.895820
22,FafduPlessis,14,475,60,36,730,160.207857,1,0.241461,33.928571,54.356237,760.987321,151.322260,0.823529,626.695441
19,DevonConway,15,481,77,18,672,117.232667,2,0.136574,32.066667,37.592608,526.296518,63.422299,0.882353,464.379281
60,RuturajGaikwad,15,400,46,30,590,139.366667,1,0.237644,26.666667,37.164444,520.302222,109.100165,0.882353,459.090196
76,ViratKohli,14,457,65,16,639,118.433571,2,0.275410,32.642857,38.660102,541.241421,122.757941,0.823529,445.728229
79,YashasviJaiswal,14,382,82,26,625,136.939286,1,0.231896,27.285714,37.364862,523.108071,99.899811,0.823529,430.794882
71,SuryakumarYadav,16,334,65,28,605,155.473750,2,0.306537,20.875000,32.455145,454.372034,131.088902,0.941176,427.644268
26,HeinrichKlaasen,11,253,32,25,448,182.172727,2,0.319339,23.000000,41.899727,586.596182,121.208976,0.647059,379.562235
57,RinkuSingh,14,317,31,29,474,144.611429,6,0.491025,22.642857,32.744159,458.418229,185.372060,0.823529,377.520894
16,DavidWarner,14,392,69,10,516,114.495000,0,0.287660,28.000000,32.058600,448.820400,106.323868,0.823529,369.616800
